In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
def import_file(file_path: str, **kwargs) -> pd.DataFrame:
    try:
        file_extension = file_path.split(".")[-1].lower()

        if file_extension == "csv":
            df = pd.read_csv(file_path, **kwargs)
        elif file_extension == "xlsx":
            df = pd.read_excel(file_path, **kwargs)
        elif file_extension == "parquet":
            df = pd.read_parquet(file_path, **kwargs)
        else:
            raise ValueError(f"Unsupported file extension: {file_extension}")

        return df

    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
        raise
    except pd.errors.ParserError as e:
        print(f"Error: Parsing error for file {file_path} - {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occured: {str(e)}")
        raise

def import_multiple_files(
    directory_path: str, extension: str, **kwargs
) -> pd.DataFrame:
    files = [f for f in os.listdir(directory_path) if f.endswith(f".{extension}")]
    dataframes = []
    row_counts = {}
    for file in files:
        file_path = os.path.join(directory_path, file)

        if extension == "csv":
            df = pd.read_csv(file_path, **kwargs)
            columns = [c for c in df.columns if c.lower()[:7] != "unnamed"]
            df = df[columns]
        elif extension == "xlsx":
            df = pd.read_excel(file_path, **kwargs)
        elif extension == "parquet":
            df = pd.read_parquet(file_path, **kwargs)
        else:
            raise ValueError(f"Unsupport file extension: {extension}")
        row_counts[file] = len(df)
        dataframes.append(df)

    concatenate_df = pd.concat(dataframes, ignore_index=True)
    row_counts["total"] = len(concatenate_df)

    return concatenate_df

def fill_lead_zeros(
    df: pd.DataFrame, column_name: str = "CustomerRefID"
) -> pd.DataFrame:
    if column_name not in df.columns:
        raise ValueError(f"The column '{column_name}' does not exist in the DataFrame.")

    df[column_name] = df[column_name].apply(
        lambda x: str(x).zfill(7) if pd.notnull(x) else x
    )

    return df

def convert_to_date(
    df: pd.DataFrame, column_names: list[str], date_format: str = "%Y-%m-%d"
) -> pd.DataFrame:
    for column_name in column_names:
        if column_name not in df.columns:
            raise ValueError(
                f"The column '{column_name}' does not exist in the DataFrame."
            )

        try:
            df[column_name] = pd.to_datetime(
                df[column_name], format=date_format, errors="coerce"
            )
        except Exception as e:
            raise ValueError(f"Error converting column '{column_name}' to date: {e}")

    return df

def process_dataframe(
    file_path: str,
    customer_id_column: str,
    date_columns: list[str],
    expected_dtype: dict,
    date_format: str = "%Y-%m-%d",
    **kwargs,
) -> pd.DataFrame:
    df = import_file(file_path, **kwargs)

    df = fill_lead_zeros(df, customer_id_column)

    df = convert_to_date(df, date_columns, date_format)
    
    validate_column_types(df, expected_dtype)

    return df

def process_dataframe_multiple_files(
    directory_path: str,
    extension: str,
    customer_id_column: str,
    date_columns: list[str],
    expected_dtype: dict,
    date_format: str = "%Y-%m-%d",
    **kwargs,
) -> pd.DataFrame:
    df = import_multiple_files(directory_path, extension, **kwargs)

    df = fill_lead_zeros(df, customer_id_column)

    df = convert_to_date(df, date_columns, date_format)
    
    validate_column_types(df, expected_dtype)

    return df

def process_dataframe_npl_list(
    file_path: str, customer_id_column: str, buddhist_data_column: str, expected_dtype: dict, **kwargs
) -> pd.DataFrame:
    df = import_file(file_path, **kwargs)

    df = buddhist_to_gregorian(df, buddhist_data_column)

    df = fill_lead_zeros(df, customer_id_column)

    df = df[['CustomerRefID', 'NPLMonth']]
    
    validate_column_types(df, expected_dtype)
    return df

def validate_column_types(df: pd.DataFrame, expected_types: dict) -> None:
    for column, expected_type in expected_types.items():
        if column not in df.columns:
            raise ValueError(f"The column 'column' does not exist in the DataFrame.")

        actual_type = df[column].dtype

        if str(actual_type) != expected_type:
            raise TypeError(
                f"Column '{column}' has data type '{actual_type}' but expected '{expected_type}'"
            )
    print("Validated!")

def buddhist_to_gregorian(df: pd.DataFrame, buddhist_data_column: str) -> pd.DataFrame:
    df["Year"] = df[buddhist_data_column].str[:4].astype("int")
    df["Month"] = df[buddhist_data_column].str[5:7].astype("int")
    df["Day"] = df[buddhist_data_column].str[8:10].astype("int")
    df["AD_Year"] = df["Year"] - 543
    df["NPLMonthAD"] = pd.to_datetime(
        dict(year=df["AD_Year"], month=df["Month"], day=df["Day"])
    )
    df.drop(["NPLMonth", "Year", "Month", "Day", "AD_Year"], axis=1, inplace=True)
    df.rename(columns={"NPLMonthAD": "NPLMonth"}, inplace=True)

    return df

def filter_good_201910_202307_data(df: pd.DataFrame) -> pd.DataFrame:
    condition_1 = (df["WorkFlowStepName"].isin(["Complete", "Final Rating"]))
    condition_2 = (df["FlowModelNew62"].isin(["E", "N1"]))
    condition_3 = (df["CompositeRate"].isin(["A", "B1", "B2", "B3", "B4", "C1", "C2", "C3"]))
    condition_4 = (df["CompositeFullScore"] > 0)
    
    # df_filtered = df[condition_1 & condition_2 & condition_3 & condition_4]
    df_filtered_1 = df[condition_1]
    print(df_filtered_1.shape)
    df_filtered_2 = df_filtered_1[condition_2]
    print(df_filtered_2.shape)
    df_filtered_3 = df_filtered_2[condition_3]
    print(df_filtered_3.shape)
    df_filtered_4 = df_filtered_3[condition_4]
    print(df_filtered_4.shape)
    df_filtered = df_filtered_4
    
    return df_filtered

def filter_good_201910_202403_data(df: pd.DataFrame) -> pd.DataFrame:
    condition_1 = (df["RatingDate"].notnull())
    condition_2 = (df["RatingDate"] == df["MaxRatingDate"])
    condition_3 = (df["CustomerRefID"] != "0000000")
    condition_4 = (df["CustomerRefID"] != "9999999")
    condition_5 = (df["CustomerRefID"].notnull())
    condition_6 = (df["WorkFlowStepName"].isin(["Complete", "Final Rating"]))
    condition_7 = (df["RatingYear"] != 0)
    condition_8 = (df["CompositeRate"].notnull())
    condition_9 = (df["CompositeScore"].notnull())
    condition_10 = (df["AfterAug23Flag"] == 0 | ((df["AfterAug23Flag"] == 1) & (df["RequestID"].str[0].isin(["C", "M"]))))
    
    df_filtered = df[condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7 & condition_8 & condition_9 & condition_10]
    
    return df_filtered

def keep_non_npl_customer(df: pd.DataFrame, npl_df: pd.DataFrame, key_mapping: str="CustomerRefID") -> pd.DataFrame:
    
    non_npl_df = df[~df[key_mapping].isin(npl_df[key_mapping])]
    
    return non_npl_df

def filter_bad_201910_202307(df: pd.DataFrame) -> pd.DataFrame:
    condition_1 = (df["ScoreModel"] == "New62")
    condition_2 = (df["CustomerRefID"] != "0000000")
    condition_3 = (df["CustomerRefID"] != "9999999")
    condition_4 = (df["CustomerRefID"].notnull())
    condition_5 = (df["FlowModelNew62"].isin(["N1", "E"]))
    condition_6 = (df["CompositeScore"] > 0)
    
    df_filtered = df[condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6]
    
    return df_filtered

def filter_bad_202308_202403(df: pd.DataFrame) -> pd.DataFrame:
    df_1 = df.copy()
    df_1["MaxRatingDate"] = df_1.groupby("RequestID")["RatingDate"].transform("max")
    
    condition_1 = (df_1["RatingDate"].notnull())
    condition_2 = (df_1["RatingDate"] == df_1["MaxRatingDate"])
    condition_3 = (df_1["CustomerRefID"] != "0000000")
    condition_4 = (df_1["CustomerRefID"] != "9999999")
    condition_5 = (df_1["CustomerRefID"].notnull())
    condition_6 = (df_1["RatingYear"] != 0)
    condition_7 = (df_1["CompositeRate"].notnull())
    condition_8 = (df_1["CompositeScore"].notnull())
    condition_9 = (df_1["RequestID"].str[0].isin(["C", "M"]))
    
    df_filtered = df_1[condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7 & condition_8 & condition_9] 
    
    return df_filtered

In [ ]:
composite_201910_202307_data_dtype = {
    "RequestID": "object",
    "RequestDate": "object",
    "RatingDate": "object",
    "RatingYear": "int64",
    "Status": "object",
    "WorkFlowStepName": "object",
    "CustomerID": "object",
    "CustomerRefID": "object",
    "CustomerName": "object",
    "BusinessSize": "object",
    "BusinessCodeMOC": "object",
    "BusinessCodeISIC": "object",
    "EXIMCustDate": "object",
    "LatestApprovalDate": "object",
    "StartDate": "object",
    "BusinessAge": "int64",
    "ScoreModel": "object",
    "FlowModelExisting": "object",
    "FlowModelNew62": "object",
    "RequestCustType": "object",
    "RegistrationID": "object",
    "Model": "object",
    "HasFinancialStatement": "object",
    "RevenueEstimate": "float64",
    "ApprovalLimit": "float64",
    "CustTypePrescreen": "object",
    "CompositeScore": "float64",
    "CompositeFullScore": "float64",
    "CompositeRate": "object",
    "PrescreenResult": "object",
    "CutoffColor": "object",
    "OverrideLevel": "float64",
    "PreApproveFinalCompositeRate": "object",
    "FinalCutoffColor": "object",
    "FinalDecisionOverrideLevel": "object",
    "FinalRate": "object",
    "FinancialScore": "float64",
    "FinancialFullScore": "float64",
    "FinancialRate": "object",
    "adjFinancialScore": "float64",
    "adjFinancialRate": "object",
    "BusinessScore": "float64",
    "BusinessFullScore": "float64",
    "BusinessRate": "object",
    "IndustryScore": "float64",
    "IndustryFullScore": "float64",
    "Mapping NPLs": "object",
}
composite_201910_202307_date_columns = [
    "RequestDate",
    "RatingDate",
    "EXIMCustDate",
    "LatestApprovalDate",
    "StartDate",
]
composite_201910_202307_expected_dtype = {
    "RequestID": "object",
    "RequestDate": "datetime64[ns]",
    "RatingDate": "datetime64[ns]",
    "RatingYear": "int64",
    "Status": "object",
    "WorkFlowStepName": "object",
    "CustomerID": "object",
    "CustomerRefID": "object",
    "CustomerName": "object",
    "BusinessSize": "object",
    "BusinessCodeMOC": "object",
    "BusinessCodeISIC": "object",
    "EXIMCustDate": "datetime64[ns]",
    "LatestApprovalDate": "datetime64[ns]",
    "StartDate": "datetime64[ns]",
    "BusinessAge": "int64",
    "ScoreModel": "object",
    "FlowModelExisting": "object",
    "FlowModelNew62": "object",
    "RequestCustType": "object",
    "RegistrationID": "object",
    "Model": "object",
    "HasFinancialStatement": "object",
    "RevenueEstimate": "float64",
    "ApprovalLimit": "float64",
    "CustTypePrescreen": "object",
    "CompositeScore": "float64",
    "CompositeFullScore": "float64",
    "CompositeRate": "object",
    "PrescreenResult": "object",
    "CutoffColor": "object",
    "OverrideLevel": "float64",
    "PreApproveFinalCompositeRate": "object",
    "FinalCutoffColor": "object",
    "FinalDecisionOverrideLevel": "object",
    "FinalRate": "object",
    "FinancialScore": "float64",
    "FinancialFullScore": "float64",
    "FinancialRate": "object",
    "adjFinancialScore": "float64",
    "adjFinancialRate": "object",
    "BusinessScore": "float64",
    "BusinessFullScore": "float64",
    "BusinessRate": "object",
    "IndustryScore": "float64",
    "IndustryFullScore": "float64",
    "Mapping NPLs": "object",
}

composite_202308_202403_data_dtype = {
    "No": "int64",
    "RequestID": "object",
    "RequestDate": "object",
    "RatingDate": "object",
    "RatingYear": "int64",
    "Status": "object",
    "WorkFlowStepName": "object",
    "CustomerID": "object",
    "CustomerRefID": "object",
    "CustomerName": "object",
    "BusinessSize": "object",
    "BusinessCodeMOC": "object",
    "BusinessCodeISIC": "object",
    "EXIMCustDate": "object",
    "LastApprovalDate": "object",
    "StartDate": "object",
    "BusinessAge": "int64",
    "ScoreModel": "object",
    "FlowModelExisting": "object",
    "FlowModelNew62": "object",
    "RequestCustType": "object",
    "RegistrationID": "object",
    "Model": "object",
    "HasFinancialStatement": "object",
    "RevenueEstimate": "float64",
    "ApprovalLimit": "float64",
    "CustType": "object",
    "JuristicType": "object",
    "CompositeScore": "float64",
    "CompositeFullScore": "float64",
    "CompositeRate": "object",
    "PrescreenResult": "object",
    "CutoffColor": "object",
    "Override": "float64",
    "PreApproveFinalCompositeRate": "object",
    "FinalCutoffColor": "object",
    "FinalDecisionOverrideLevel": "object",
    "FinalRate": "object",
    "FinancialScore": "float64",
    "FinancialFullScore": "float64",
    "FinancialRate": "object",
    "adjFinancialScore": "float64",
    "adjFinancialRate": "object",
    "BusinessScore": "float64",
    "BusinessFullScore": "float64",
    "BusinessRate": "object",
    "IndustryScore": "float64",
    "IndustryFullScore": "float64",
}
composite_202308_202403_date_columns = [
    "RequestDate",
    "RatingDate",
    "EXIMCustDate",
    "LastApprovalDate",
    "StartDate",
]
composite_202308_202403_expected_dtype = {
    "No": "int64",
    "RequestID": "object",
    "RequestDate": "datetime64[ns]",
    "RatingDate": "datetime64[ns]",
    "RatingYear": "int64",
    "Status": "object",
    "WorkFlowStepName": "object",
    "CustomerID": "object",
    "CustomerRefID": "object",
    "CustomerName": "object",
    "BusinessSize": "object",
    "BusinessCodeMOC": "object",
    "BusinessCodeISIC": "object",
    "EXIMCustDate": "datetime64[ns]",
    "LastApprovalDate": "datetime64[ns]",
    "StartDate": "datetime64[ns]",
    "BusinessAge": "int64",
    "ScoreModel": "object",
    "FlowModelExisting": "object",
    "FlowModelNew62": "object",
    "RequestCustType": "object",
    "RegistrationID": "object",
    "Model": "object",
    "HasFinancialStatement": "object",
    "RevenueEstimate": "float64",
    "ApprovalLimit": "float64",
    "CustType": "object",
    "JuristicType": "object",
    "CompositeScore": "float64",
    "CompositeFullScore": "float64",
    "CompositeRate": "object",
    "PrescreenResult": "object",
    "CutoffColor": "object",
    "Override": "float64",
    "PreApproveFinalCompositeRate": "object",
    "FinalCutoffColor": "object",
    "FinalDecisionOverrideLevel": "object",
    "FinalRate": "object",
    "FinancialScore": "float64",
    "FinancialFullScore": "float64",
    "FinancialRate": "object",
    "adjFinancialScore": "float64",
    "adjFinancialRate": "object",
    "BusinessScore": "float64",
    "BusinessFullScore": "float64",
    "BusinessRate": "object",
    "IndustryScore": "float64",
    "IndustryFullScore": "float64",
}

npl_list_data_dtype = {"CustomerRefID": str, "NPLMonth": str}
npl_list_expected_dtype = {"CustomerRefID": "object", "NPLMonth": "datetime64[ns]"}

In [ ]:
corp_sme_composite_201910_202307_data_path = "../data/raw/Corporate_and_SMEs/Composite_Parallel_Run_Data/Scoreresult_Composite_2019-2023_mapping_npl.xlsx"
corp_sme_composite_202308_202403_data_path =  "../data/raw/Corporate_and_SMEs/Composite_Ongoing_Data/"
npl_list_path = "../data/raw/NPLs/NPL_list.xlsx"

In [ ]:
composite_201910_202307_df = process_dataframe(
    file_path=corp_sme_composite_201910_202307_data_path,
    customer_id_column="CustomerRefID",
    expected_dtype=composite_201910_202307_expected_dtype,
    date_columns=composite_201910_202307_date_columns,
    dtype=composite_201910_202307_data_dtype,
)
composite_201910_202307_df.shape
# (39034, 47)

In [ ]:
composite_202308_202403_df = process_dataframe_multiple_files(
    directory_path=corp_sme_composite_202308_202403_data_path,
    extension="csv",
    customer_id_column="CustomerRefID",
    expected_dtype=composite_202308_202403_expected_dtype,
    date_columns=composite_202308_202403_date_columns,
    dtype=composite_202308_202403_data_dtype,
)
composite_202308_202403_df.shape
# (5703, 48)

In [ ]:
npl_list_df = process_dataframe_npl_list(
    file_path=npl_list_path,
    customer_id_column="CustomerRefID",
    buddhist_data_column="NPLMonth",
    expected_dtype=npl_list_expected_dtype,
    dtype=npl_list_data_dtype,
)
npl_list_df.shape
# (637, 2)

In [ ]:
relevant_columns = [
    "RequestID",
    "RequestDate",
    "RatingDate",
    "RatingYear",
    "WorkFlowStepName",
    "CustomerRefID",
    "ScoreModel",
    "FlowModelExisting",
    "FlowModelNew62",
    "CompositeScore",
    "CompositeFullScore",
    "CompositeRate",
    "adjFinancialScore",
    "BusinessScore",
    "IndustryScore",
    "AfterAug23Flag",
    "Mapping NPLs",
]

In [ ]:
# Add new columns for data preparation
composite_202308_202403_df["Mapping NPLs"] = np.NaN
composite_202308_202403_df["AfterAug23Flag"] = 1
composite_201910_202307_df["AfterAug23Flag"] = 0

# Select relevant columns
base_201910_202307_df = composite_201910_202307_df[relevant_columns]
base_202308_202403_df = composite_202308_202403_df[relevant_columns]

# (5091, 12)
base_good_201910_202307_df = filter_good_201910_202307_data(base_201910_202307_df)
# (5703, 12)
base_good_202308_202403_df = base_202308_202403_df.copy()
# (10794, 12)
base_good_201910_202403_df = pd.concat(
    [base_good_201910_202307_df, base_good_202308_202403_df], ignore_index=True
)
# (10709, 12)
base_good_201910_202403_df_1 = base_good_201910_202403_df.drop_duplicates(
    subset=[
        "RequestID",
        "RequestDate",
        "RatingDate",
        "RatingYear",
        "WorkFlowStepName",
        "CustomerRefID",
    ]
).copy()

# Create MaxRatingDate column for filtering data (select latest observation for each RequestID)
base_good_201910_202403_df_1["MaxRatingDate"] = base_good_201910_202403_df_1.groupby("RequestID")["RatingDate"].transform("max")
# (7896, 13)
base_good_201910_202403_df_2 = filter_good_201910_202403_data(base_good_201910_202403_df_1)
# (7621, 13)
base_good_201910_202403_df_3 = keep_non_npl_customer(base_good_201910_202403_df_2, npl_list_df).copy()

base_good_201910_202403_df_3["NPLMonth"] = pd.NaT
base_good_201910_202403_df_3["NextRatingDate"] = base_good_201910_202403_df_3.sort_values(["CustomerRefID", "RatingDate"]).groupby("CustomerRefID")["RatingDate"].shift(-1)
base_good_201910_202403_df_3["NPLMonthAdj"] = pd.NaT
base_good_201910_202403_df_3["DefaultFlag"] = 0
base_good_201910_202403_df_3["MaxDefaultFlag"] = 0

base_good_1_201910_202403_df_corp_tmp = base_good_201910_202403_df_3[base_good_201910_202403_df_3["RequestID"].str[0].isin(["C", "N"])].copy()
base_good_1_201910_202403_df_sme_tmp = base_good_201910_202403_df_3[base_good_201910_202403_df_3["RequestID"].str[0].isin(["M", "E"])].copy()

# Manual Adjust
base_good_1_201910_202403_df_sme = base_good_1_201910_202403_df_sme_tmp[~base_good_1_201910_202403_df_sme_tmp["RequestID"].isin(["E20120016", "M23090003"])]

ManualAdjustData = base_good_1_201910_202403_df_sme_tmp[base_good_1_201910_202403_df_sme_tmp["RequestID"].isin(["E20120016", "M23090003"])]
ManualAdjustDataFinal = ManualAdjustData.copy()

base_good_1_201910_202403_df_corp = pd.concat([base_good_1_201910_202403_df_corp_tmp, ManualAdjustDataFinal], ignore_index=True)

In [ ]:
# (6459, 12)
base_bad_201910_202307_df = filter_bad_201910_202307(base_201910_202307_df)
# (3704, 12)
base_bad_202308_202403_df = filter_bad_202308_202403(base_202308_202403_df)

# Concatenate base_bad data
base_bad_201910_202403_df = pd.concat([base_bad_201910_202307_df, base_bad_202308_202403_df], ignore_index=True)
# No duplicate drop
base_bad_201910_202403_df_1 = base_bad_201910_202403_df.drop_duplicates(
    subset=[
        "RequestID",
        "RequestDate",
        "RatingDate",
        "RatingYear",
        "WorkFlowStepName",
        "CustomerRefID",
    ]
).copy()

# (379, 14)
base_bad_201910_202403_df_2 = base_bad_201910_202403_df_1[base_bad_201910_202403_df_1["CustomerRefID"].isin(npl_list_df["CustomerRefID"])]
base_bad_201910_202403_df_3 = base_bad_201910_202403_df_2.merge(npl_list_df, how='left', on=['CustomerRefID']).sort_values(["CustomerRefID", "RequestID", "RatingYear", "RatingDate"])
base_bad_201910_202403_df_3["NPLMonth"] == base_bad_201910_202403_df_3["NPLMonth"] + pd.offsets.MonthEnd(0)
base_bad_201910_202403_df_3["NextRatingDate"] = base_bad_201910_202403_df_3.sort_values(["CustomerRefID", "RatingDate"]).groupby("CustomerRefID")["RatingDate"].shift(-1)
base_bad_201910_202403_df_3["NPLMonthAdj"] = pd.to_datetime(
    np.where(
        base_bad_201910_202403_df_3["NPLMonth"].between(
            base_bad_201910_202403_df_3["RatingDate"], base_bad_201910_202403_df_3["NextRatingDate"], inclusive="left"
        ),
        base_bad_201910_202403_df_3["NPLMonth"],
        np.where(
            (base_bad_201910_202403_df_3["NextRatingDate"].isnull())
            & (base_bad_201910_202403_df_3["NPLMonth"] >= base_bad_201910_202403_df_3["RatingDate"]),
            base_bad_201910_202403_df_3["NPLMonth"],
            pd.NaT,
        ),
    )
)
base_bad_201910_202403_df_3.sort_values(["RequestID", "NPLMonthAdj"], inplace=True)
base_bad_201910_202403_df_3["row_num"] = base_bad_201910_202403_df_3.groupby(["CustomerRefID", "RequestID"]).cumcount() + 1
base_bad_201910_202403_df_4 = base_bad_201910_202403_df_3.query("row_num == 1").copy()
base_bad_201910_202403_df_4["DefaultFlag"] = np.where(base_bad_201910_202403_df_4["NPLMonthAdj"].notnull(), 1, 0)
base_bad_201910_202403_df_4.drop(["row_num"], axis=1, inplace=True)

# Split Portfolio
base_bad_201910_202403_df_corp = base_bad_201910_202403_df_4[base_bad_201910_202403_df_4["RequestID"].str[0].isin(["C", "N"])].copy()
base_bad_201910_202403_df_corp["MaxDefaultFlag"] = base_bad_201910_202403_df_corp.groupby("CustomerRefID")["DefaultFlag"].transform("max")
base_bad_201910_202403_df_sme = base_bad_201910_202403_df_4[base_bad_201910_202403_df_4["RequestID"].str[0].isin(["M", "E"])].copy()
base_bad_201910_202403_df_sme["MaxDefaultFlag"] = base_bad_201910_202403_df_sme.groupby("CustomerRefID")["DefaultFlag"].transform("max")

# Good Customers
base_good_2_201910_202403_df_corp = base_bad_201910_202403_df_corp.query("MaxDefaultFlag == 0 & WorkFlowStepName.isin(['Complete', 'Final Rating'])").copy()
base_good_2_201910_202403_df_sme = base_bad_201910_202403_df_sme.query("MaxDefaultFlag == 0 & WorkFlowStepName.isin(['Complete', 'Final Rating'])").copy()
# Bad Customers
base_bad_201910_202403_df_corp_1 = base_bad_201910_202403_df_corp.query("MaxDefaultFlag == 1").copy()
base_bad_201910_202403_df_sme_1 = base_bad_201910_202403_df_sme.query("MaxDefaultFlag == 1").copy()

In [ ]:
# Finalize data
base_good_201910_202403_corp_final = pd.concat([base_good_1_201910_202403_df_corp , base_good_2_201910_202403_df_corp], ignore_index=True)
base_good_201910_202403_sme_final = pd.concat([base_good_1_201910_202403_df_sme , base_good_2_201910_202403_df_sme], ignore_index=True)

base_bad_201910_202403_corp_final = base_bad_201910_202403_df_corp_1.copy()
base_bad_201910_202403_sme_final = base_bad_201910_202403_df_sme_1.copy()

base_201910_202307_corp_final = pd.concat([base_good_201910_202403_corp_final, base_bad_201910_202403_corp_final], ignore_index=True)
base_201910_202307_sme_final = pd.concat([base_good_201910_202403_sme_final, base_bad_201910_202403_sme_final], ignore_index=True)

In [ ]:
print(base_201910_202307_corp_final.shape)
print(base_201910_202307_sme_final.shape)

In [ ]:
corp_output_path = '../data/processed/01_master_data/base_201910_202307_corporate.parquet'
sme_output_path = '../data/processed/01_master_data/base_201910_202307_sme.parquet'
corp_csv_output_path = '../data/processed/01_master_data/base_201910_202307_corporate.csv'
sme_csv_output_path = '../data/processed/01_master_data/base_201910_202307_sme.csv'

base_201910_202307_corp_final.to_parquet(corp_output_path)
base_201910_202307_sme_final.to_parquet(sme_output_path)
base_201910_202307_corp_final.to_csv(corp_csv_output_path)
base_201910_202307_sme_final.to_csv(sme_csv_output_path)

In [ ]:
# Check RequestID uniqueness
print(base_201910_202307_corp_final["RequestID"].value_counts().reset_index().query("count > 1"))
print(base_201910_202307_sme_final["RequestID"].value_counts().reset_index().query("count > 1"))

# Check data duplicate
print(base_201910_202307_corp_final.drop_duplicates().shape)
print(base_201910_202307_sme_final.drop_duplicates().shape)